In [1]:
import os
import glob
import json
import nltk
from nltk import sent_tokenize
from nltk.stem import WordNetLemmatizer

import tqdm

In [22]:
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('tagsets')
# nltk.download('stopword')
# nltk.download('maxent_treebank_pos_tagger')
# nltk.download('averaged_perceptron_tagger')

In [3]:
os.getcwd()

'C:\\Users\\Chankoo\\Desktop\\GitHub\\BOAZ-projects\\airbnb-NLP'

In [4]:
os.chdir('./../airbnb-data')

In [8]:
file_lst = glob.glob('review_*_*.json')
file_lst

['review_busan_70028.json',
 'review_daegu_33689.json',
 'review_jeju_143957.json',
 'review_jeonju_13647.json',
 'review_seogwipo_51728.json',
 'review_seoul_326990.json']

In [9]:
review = {}
for file in file_lst:
    with open(file,'r',encoding='utf-8') as fp:
        review.update(json.load(fp))

In [10]:
len(review)

28853

In [16]:
review_en = {}
for home_id,rev_lst in review.items():
    review_en[home_id] = []
    for rev in rev_lst:
        if rev['language']=='en':
            review_en[home_id].append([rev['rating'],rev['comments']])
        else:
            continue

In [17]:
review_en['10008511']

[[4,
  "It's a great experience staying at Hanuel's house. It's clean, kitchen is equipped with basic needs. Finding this place isn't hard as well, and it's really a good point that it's close to the airport.\r\n\r\nThere's some really good restaurant around the house. We had a good bbq duck meal and it's awesome!\r\n\r\nWould recommend this to anyone who like to visit Jeju!"]]

In [18]:
sent_tokenize(review_en['10008511'][0][1])

["It's a great experience staying at Hanuel's house.",
 "It's clean, kitchen is equipped with basic needs.",
 "Finding this place isn't hard as well, and it's really a good point that it's close to the airport.",
 "There's some really good restaurant around the house.",
 "We had a good bbq duck meal and it's awesome!",
 'Would recommend this to anyone who like to visit Jeju!']

In [19]:
%%time
# 리뷰별 문장단위 tokenizing
for home_id,rev_lst in review_en.items():
    for rev in rev_lst:
        rev[1] = sent_tokenize(rev[1])

Wall time: 31.6 s


In [27]:
%%time 
# 문장별 단어단위 tokenizing
for home_id,rev_lst in review_en.items(): 
    for rev in rev_lst:
        tmp_lst = []
        for sent in rev[1]:
            word_lst = nltk.word_tokenize(sent)
            tmp_lst.append(word_lst)
        rev[1] = tmp_lst

In [26]:
nltk.pos_tag(review_en['10008511'][0][1][0])

[('It', 'PRP'),
 ("'s", 'VBZ'),
 ('a', 'DT'),
 ('great', 'JJ'),
 ('experience', 'NN'),
 ('staying', 'VBG'),
 ('at', 'IN'),
 ('Hanuel', 'NNP'),
 ("'s", 'POS'),
 ('house', 'NN'),
 ('.', '.')]

In [29]:
for home_id,rev_lst in tqdm.tqdm(review_en.items()):
    for rev in rev_lst:
        tmp_lst = []
        for sent_bow in rev[1]:
            sent_pos = nltk.pos_tag(sent_bow)
            tmp_lst.append(sent_pos)
        rev[1] = tmp_lst

100%|████████████████████████████████████████████████████████████████████████████| 28853/28853 [27:01<00:00, 17.80it/s]


### 데이터 구조를 보자

- review_en == {home_id: [rev0, rev1, ...]}
    - rev0 == [rating, [sent0, sent1, ...]]
        - sent0 == [(word0,POS0),(word1,POS1), ...]
    

In [30]:
review_en['10022410'][0] # rev0의 예시

[4,
 [[('Kyungsoon', 'NNP'),
   ('’', 'NNP'),
   ('s', 'JJ'),
   ('place', 'NN'),
   ('is', 'VBZ'),
   ('located', 'VBN'),
   ('at', 'IN'),
   ('a', 'DT'),
   ('quiet', 'JJ'),
   ('and', 'CC'),
   ('peaceful', 'JJ'),
   ('neighborhood', 'NN'),
   ('not', 'RB'),
   ('far', 'RB'),
   ('from', 'IN'),
   ('the', 'DT'),
   ('city', 'NN'),
   ('.', '.')],
  [('Recommended', 'VBN'),
   ('to', 'TO'),
   ('those', 'DT'),
   ('who', 'WP'),
   ('have', 'VBP'),
   ('rent', 'VBN'),
   ('a', 'DT'),
   ('car', 'NN'),
   (',', ','),
   ('the', 'DT'),
   ('parking', 'VBG'),
   ('area', 'NN'),
   ('is', 'VBZ'),
   ('spacious', 'JJ'),
   ('.', '.')],
  [('The', 'DT'),
   ('house', 'NN'),
   ('is', 'VBZ'),
   ('clean', 'JJ'),
   ('and', 'CC'),
   ('comfortable', 'JJ'),
   ('.', '.')],
  [('We', 'PRP'),
   ('are', 'VBP'),
   ('a', 'DT'),
   ('group', 'NN'),
   ('of', 'IN'),
   ('4', 'CD'),
   (',', ','),
   ('the', 'DT'),
   ('house', 'NN'),
   ('is', 'VBZ'),
   ('just', 'RB'),
   ('nice', 'JJ'),
   ('for'

unpacking과 zip 함수 이용해 rating과 review를 각각을 뽑아 리스트로 만들 수도 있다

In [81]:
rating_lst, rev_lst = zip(*review_en['10022410'])

In [89]:
rating_lst[:5]

(4, 5, 5, 5, 5)

In [85]:
rev_lst[0]

[[('Kyungsoon', 'NNP'),
  ('’', 'NNP'),
  ('s', 'JJ'),
  ('place', 'NN'),
  ('is', 'VBZ'),
  ('located', 'VBN'),
  ('at', 'IN'),
  ('a', 'DT'),
  ('quiet', 'JJ'),
  ('and', 'CC'),
  ('peaceful', 'JJ'),
  ('neighborhood', 'NN'),
  ('not', 'RB'),
  ('far', 'RB'),
  ('from', 'IN'),
  ('the', 'DT'),
  ('city', 'NN'),
  ('.', '.')],
 [('Recommended', 'VBN'),
  ('to', 'TO'),
  ('those', 'DT'),
  ('who', 'WP'),
  ('have', 'VBP'),
  ('rent', 'VBN'),
  ('a', 'DT'),
  ('car', 'NN'),
  (',', ','),
  ('the', 'DT'),
  ('parking', 'VBG'),
  ('area', 'NN'),
  ('is', 'VBZ'),
  ('spacious', 'JJ'),
  ('.', '.')],
 [('The', 'DT'),
  ('house', 'NN'),
  ('is', 'VBZ'),
  ('clean', 'JJ'),
  ('and', 'CC'),
  ('comfortable', 'JJ'),
  ('.', '.')],
 [('We', 'PRP'),
  ('are', 'VBP'),
  ('a', 'DT'),
  ('group', 'NN'),
  ('of', 'IN'),
  ('4', 'CD'),
  (',', ','),
  ('the', 'DT'),
  ('house', 'NN'),
  ('is', 'VBZ'),
  ('just', 'RB'),
  ('nice', 'JJ'),
  ('for', 'IN'),
  ('us', 'PRP'),
  ('.', '.')],
 [('The', 'DT'),

In [65]:
# nltk.help.upenn_tagset()

명사, 형용사, 동사, 부사를 뽑아 lemmantization 시켜주자

함수 njvr_lemmantizer를 정의한다


In [67]:
lemm = WordNetLemmatizer()
def njvr_lemmantizer(sent):
    global lemm
    
    lemm_sent = []
    for word_pos in sent:
        word,pos=word_pos
        if pos[0] == 'N':
            lemm_sent.append(lemm.lemmatize(word,pos='n')+'_N')
        elif pos[0] == 'J':
            lemm_sent.append(lemm.lemmatize(word,pos='a')+'_J')
        elif pos[0] == 'V':
            lemm_sent.append(lemm.lemmatize(word,pos='v')+'_V')
        elif pos[0] == 'R':
            lemm_sent.append(lemm.lemmatize(word,pos='r')+'_R')
        else:pass
    return lemm_sent

샘플 sentence

In [63]:
review_en['10022410'][1][1][0]

[('We', 'PRP'),
 ('had', 'VBD'),
 ('a', 'DT'),
 ('wonderful', 'JJ'),
 ('stay', 'NN'),
 ('at', 'IN'),
 ('Kyungsoon', 'NNP'),
 ('’', 'NNP'),
 ('s', 'JJ'),
 ('place', 'NN'),
 (',', ','),
 ('although', 'IN'),
 ('he', 'PRP'),
 ('can', 'MD'),
 ('only', 'RB'),
 ('speak', 'VB'),
 ('a', 'DT'),
 ('little', 'JJ'),
 ('of', 'IN'),
 ('English', 'JJ'),
 ('but', 'CC'),
 ('he', 'PRP'),
 ("'s", 'VBZ'),
 ('always', 'RB'),
 ('trying', 'VBG'),
 ('to', 'TO'),
 ('help', 'VB'),
 ('whatever', 'VB'),
 ('he', 'PRP'),
 ('can', 'MD'),
 ('.', '.')]

lemmantization 적용후

In [68]:
njvr_lemmantizer(review_en['10022410'][1][1][0])

['have_V',
 'wonderful_J',
 'stay_N',
 'Kyungsoon_N',
 '’_N',
 's_J',
 'place_N',
 'only_R',
 'speak_V',
 'little_J',
 'English_J',
 "'s_V",
 'always_R',
 'try_V',
 'help_V',
 'whatever_V']

전체 리뷰에 적용하자

In [88]:
for listing_id, rev_lst in tqdm.tqdm(review_en.items()):
    for rev in rev_lst:
        rev_tmp = []
        for sent in rev[1]:
            rev_tmp.append(njvr_lemmantizer(sent))
        rev[1] = rev_tmp

In [91]:
list(review_en.items())[1][1][1]

[5,
 [['be_V', 'great_J', 'experience_N', 'stay_V', 'Jin_N', 'home_N'],
  ['Jin_N', 'mother_N', 'brother_N', 'be_V', 'so_R', 'good_J', 'friendly_J'],
  ['be_V', 'feel_V', 'be_V', 'stay_V', 'family_N'],
  ['Jin_N',
   'mother_N',
   'have_V',
   'wide_J',
   'variety_N',
   'cook_V',
   'utensil_N',
   'beautiful_J',
   'kitchen_N'],
  ['Sometimes_R', 'offer_V', 'tasty_J', 'Korean_J', 'food_N'],
  ['Location_N',
   'home_N',
   'be_V',
   'situate_V',
   'minute_N',
   'walk_V',
   'distance_N',
   'Pusan_N',
   'national_J',
   'university_N'],
  ['be_V', 'so_R', 'many_J', 'mart_N', 'nearby_R', 'apartment_N'],
  ['Last_J', 'not_R', 'least_J', 'miss_V', 'Jin_N', 'family_N', 'home_N']]]

In [92]:
with open('review_en.json','w') as fp:
    json.dump(review_en,fp)